In [129]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats
import numpy as np

In [130]:
N = 50
coeff = [2, 3, -2, 1, 1, -1]
alpha = 0.05

sample = stats.uniform.rvs (size = N * 5) * 2 - 1
sample = sample.reshape (N, 5)
etta = np.zeros (N)

for i in range (N):
    etta [i] = np.random.normal (coeff[0] + coeff[1:] @ sample[i].T, 1.5)

### Вспомогательные функции

In [131]:
def getTSS (y):
    return sum ([ (y_i - np.mean(y))**2 for y_i in y])

def getRegression (psi, Y):
    F = psi.T @ psi
    F_inverse = np.linalg.inv (F)
    
    betta = (F_inverse @ psi.T) @ Y
    
    e = Y - (psi @ betta)
    RSS = e.T @ e
    TSS = getTSS (Y)
    R = (TSS - RSS) / TSS
    
    return betta, RSS, TSS, R, e

### a) Проверить переменные кси на мультиколлинеарность

In [132]:
for i in range (5):
    psi_xi = np.copy (sample)
    for j in range (N):
        psi_xi[j][i] = 1
    
    regression_xi = getRegression (psi_xi, sample.T[i])
    R = regression_xi [3]
    
    print ("Для кси {i} R^2 = {R}".format (i = i, R = R))
    if (R < 0.7):
        print ("Не выбрасываем.")
    else:
        print ("Выбрасываем")

Для кси 0 R^2 = 0.017587152933845687
Не выбрасываем.
Для кси 1 R^2 = 0.006167110646810398
Не выбрасываем.
Для кси 2 R^2 = 0.016313949626571633
Не выбрасываем.
Для кси 3 R^2 = 0.006809910502129443
Не выбрасываем.
Для кси 4 R^2 = 0.01627998940436859
Не выбрасываем.


### b) Определить уравнение линейной регрессии и проверить значимость коэффициентов

In [133]:
# Добавляем к уравнению константу
tmp_sample = np.copy (sample)
psi = np.zeros ((N, 6))

for i in range (N):
    psi [i] = np.insert (tmp_sample[i], 0, 1)

regression = getRegression (psi, etta)

print ("Уравнение линейной регрессии: {betta}".format(betta = regression[0]))

Уравнение линейной регрессии: [ 1.95061455  2.95002048 -1.54122213  0.96691799  0.1789411  -0.92927573]


In [134]:
F_inverse = np.linalg.inv (psi.T @ psi)
RSS = regression [1]

betta_p_value = np.zeros (6)

for i in range (6):
    delta_up = regression[0][i] * np.sqrt (N - 6)
    delta_down = np.sqrt (RSS * F_inverse[i][i])
    
    delta = delta_up / delta_down
    
    p_value = 2 * (1 - stats.t.cdf (np.abs(delta), N - 6))
    betta_p_value [i] = p_value
    print ("Для коэффициента {i} p-value = {p_value}".format (i = i, p_value = p_value))
    if (p_value > alpha):
        print ("Не значим.")
    else:
        print ("Значим.")

Для коэффициента 0 p-value = 1.0011591555780797e-10
Значим.
Для коэффициента 1 p-value = 1.3945373744661538e-09
Значим.
Для коэффициента 2 p-value = 0.00015528659240660048
Значим.
Для коэффициента 3 p-value = 0.019078771306177078
Значим.
Для коэффициента 4 p-value = 0.6200962116955391
Не значим.
Для коэффициента 5 p-value = 0.011438430772191532
Значим.


### с) Определить коэффициент детерминации и проверить его значимость

In [135]:
R = regression [3]
print ("Коэффициент детерминации:", R)

delta = (R / (1 - R)) * (N - 6) / 5
p_value = 1 - stats.f.cdf (np.abs (delta), 5, N - 6)

print ("p-value коэффициента детерминации:", p_value)

if (p_value > alpha):
    print ("Регрессия не значима.")
else:
    print ("Регрессия значима.")

Коэффициент детерминации: 0.6693966601241005
p-value коэффициента детерминации: 1.2680196892489448e-09
Регрессия значима.


### d) Найти значение в точке xk = 0 и построить 95% доверительный интервал

In [136]:
betta = regression [0]

x_0 = np.zeros (5)
psi_x0 = np.insert (x_0, 0, 1)

etta_0 = np.random.normal (coeff[0] + coeff[1:] @ x_0.T, 1.5)
y_0 = betta[0] + betta[1:] @ x_0.T

print ("Значение в точке xk = 0:", y_0)

delta = np.sqrt ( (RSS * (1 + (psi_x0 @ F_inverse) @ psi_x0.T)) / (N - 6) )
t = stats.t.ppf (1.95 / 2, N - 6)
print ("Доверительный интервал: [{left}, {right}]".format (left = y_0 - t * delta, right = y_0 + t * delta))

Значение в точке xk = 0: 1.9506145467551188
Доверительный интервал: [-1.2503773170244714, 5.1516064105347095]


### e) Проверить независимость ошибок измерения

In [137]:
e = regression [4]
J = 0

for i in range (N):
    for j in range (i + 1, N):
        if (e[i] > e[j]):
            J += 1

delta = (J - N * (N - 1) / 4) / np.sqrt (N ** 3 / 36)
p_value = 2 * (1 - stats.norm.cdf (np.abs (delta)))

print ("p-value:", p_value)
if (p_value < alpha):
    print ("Отвергаем гипотезу о независимости ошибок измерения.")
else:
    print ("Нет оснований отвергнуть гипотезу о независимости ошибок измерения.")

p-value: 0.6900345164260671
Нет оснований отвергнуть гипотезу о независимости ошибок измерения.


### f) Проверить нормальность распределения ошибок

### h) Кросс-проверка регрессии

In [138]:
TSS = regression [2]
CVSS = 0

for i in range (N):
    etta_cv = np.delete (etta, i, axis = 0)
    
    psi_cv = np.empty ([N - 1, 6])
    tmp = np.delete (sample, i, axis = 0)
    for i in range (N - 1):
        psi_cv [i] = np.insert (tmp [i], 0, 1)
    
    F_cv = psi_cv.T @ psi_cv
    F_cv_inverse = np.linalg.inv (F_cv)
    betta_cv = (F_cv_inverse @ psi_cv.T) @ etta_cv.T
    
    CVSS += (etta [i] - (betta_cv[0] + betta_cv[1:] @ sample[i]))**2

R_cv = (TSS - CVSS) / TSS
print ("Кросс-проверка - R^2:", R_cv)

if (R_cv < 0.7):
    print ("Модель плохо предсказывает.")
else:
    print ("Модель нормально предсказывает.")

Кросс-проверка - R^2: 0.9937253735384026
Модель нормально предсказывает.


### i) Проверить адекватность регрессии

In [139]:
x = sample [0]
y = np.zeros (5)

for i in range (5):
    y [i] = np.random.normal (coeff [0] + coeff [1:] @ x.T, 1.5)

sigma = (1/4) * sum ( (y_i - np.mean (y))**2 for y_i in y)
delta = (RSS / (N - 6)) * sigma

p_value = 1 - stats.f.cdf (np.abs (delta), 4, 5)
print (p_value)

if (p_value < 0.05):
    print ("Модель в неадеквате.")
else:
    print ("Модель в адеквате.")

0.3219316848238831
Модель в адеквате.


### j) Удалить малозначащую переменную и повторить пункты b и c, сделать сравнение

In [143]:
print ("Самая малозначимая переменная:", np.argmax (betta_p_value) - 1)

psi_deleted = np.delete (psi, np.argmax (betta_p_value), axis = 1)
regression_deleted = getRegression (psi_deleted, etta)

print ("Коэффициенты новой регрессии:", regression_deleted[0])

Самая малозначимая переменная: 2
Коэффициенты новой регрессии: [ 1.909435    2.89675041 -1.59215914  0.22488975 -0.86455958]


In [144]:
F_deleted = psi_deleted.T @ psi_deleted
F_inverse_deleted = np.linalg.inv (F_deleted)
for i in range (5):
    delta_up = regression_deleted[0][i] * np.sqrt (N - 5)
    delta_down = np.sqrt (regression_deleted[1] * F_inverse_deleted[i][i])
    
    delta = delta_up / delta_down
    
    p_value = 2 * (1 - stats.t.cdf (np.abs(delta), N - 5))
    betta_p_value [i] = p_value
    print ("Для коэффициента {i} p-value = {p_value}".format (i = i, p_value = p_value))
    if (p_value > alpha):
        print ("Не значим.")
    else:
        print ("Значим.")


Для коэффициента 0 p-value = 5.703189032146838e-10
Значим.
Для коэффициента 1 p-value = 6.730840773627733e-09
Значим.
Для коэффициента 2 p-value = 0.0001901473402956544
Значим.
Для коэффициента 3 p-value = 0.553810704292744
Не значим.
Для коэффициента 4 p-value = 0.023876779486845523
Значим.


In [146]:
R_deleted = regression_deleted [3]
print ("Коэффициент детерминации:", R)

delta = (R_deleted / (1 - R_deleted)) * (N - 5) / 4
p_value = 1 - stats.f.cdf (np.abs (delta), 4, N - 5)

print ("p-value коэффициента детерминации:", p_value)

if (p_value > alpha):
    print ("Регрессия не значима.")
else:
    print ("Регрессия значима.")

Коэффициент детерминации: 0.6248968734804499
p-value коэффициента детерминации: 3.946576621061126e-09
Регрессия значима.


In [149]:
RSS_deleted = regression_deleted [1]
delta_regression = (RSS_deleted - RSS) * (N - 6) / RSS

p_value = 1 - stats.f.cdf (np.abs (delta), 1, N - 6)
print (p_value)

8.509037922443952e-05
